<a href="https://colab.research.google.com/github/ashley-ferreira/PHYS449_FinalProject/blob/main/data_loading_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook to load the data in google colab:

In [ ]:
#Import Modules:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os

#Import google drive (need to put data folder as shortcut in your local drive My Drive):
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#LOAD THE DATA FROM TXT FILE INTO A BATCH:
def data_batch(datafile_index, num_images=10, data_file='/content/drive/MyDrive/data/data_g_band.txt'):
    '''
    Description:
        Access datafile.txt, each row is flattened 110x110 image + 1 label string (E, Sp, S0, Irr+Misc).
        Returns an augmented batch of num_images X 40.
        The labels are converted to 1D vectors (ex: Sp = [0,0,1,0])
        Need to give a datafile_index that tells which rows to pick.
    Inputs:
        datafile_index: index of row in datafile to load. loads rows datafile_index to datafile_index+num_images.
        num_images: number of different images to load per batch, total batch size 
        is 40 X num_images. (default: 10 (for 40X10 = 400 batch size like in paper)
        data_file: datafile full path, need to add shortcut to local Drive. (default: '/content/drive/MyDrive/data/data_g_band.txt')
    Outputs:
        tensor_input_batch_aug: dimensions: (100, 100, num_images X 40). 
        tensor_label_batch_aug: dimensions: (num_images X 40, 4)
    '''

    #data_file = 'data_g_band.txt'

    #Take batch of num_images rows from datafile:
    with open(data_file, 'r') as f:
        rows = f.readlines()[datafile_index:(datafile_index+num_images)]

    #for batch size of 400 (augmented), need 10 images
    data_batch = np.zeros((num_images,12101), dtype=np.dtype('U10'))
    count = 0
    for row in rows:
        data_batch[count,:] = row.split()
        count += 1

    #separate label and input:
    input_batch_flat = np.array(data_batch[:,:12100], dtype=int)
    label_batch = np.array(data_batch[:,-1])

    #convert input batch back to a 2D array:
    input_batch = np.empty((110,110,np.shape(input_batch_flat)[0]), dtype=int)
    for ii in range(np.shape(input_batch_flat)[0]):
        input_batch[:,:,ii] = np.reshape(input_batch_flat[ii,:], (110,110))


    #convert label batch into into 1D vector: 
    #E=0, S0=1, Sp=2, Irr+Misc=3
    #ex: label = [0,0,1,0] ==> Sp galagy
    arr_label_batch = np.empty((np.shape(label_batch)[0],4), dtype=int)
    arr_label_batch[:,0] = np.array([label_batch == 'E'], dtype=int)
    arr_label_batch[:,1] = np.array([label_batch == 'Sp'], dtype=int)
    arr_label_batch[:,2] = np.array([label_batch == 'S0'], dtype=int)
    arr_label_batch[:,3] = np.array([label_batch == 'Irr+Misc'], dtype=int)

    #test with image plotted
    #import matplotlib.pyplot as plt
    #plt.imshow(input_batch[:,:,0])
    #plt.show()

    #NOW AUGMENT THE BATCH (40X more):
    input_batch_aug = np.empty((100,100,np.shape(input_batch)[2]*40), dtype=int)
    arr_label_batch_aug = np.empty((np.shape(arr_label_batch)[0]*40, 4), dtype=int)

    count = 0
    for ll in range(np.shape(input_batch)[2]):
        #Crop 5X more image (100X100 pixels)
        C1 = input_batch[:100,:100,ll]
        C2 = input_batch[10:,:100,ll]
        C3 = input_batch[:100,10:,ll]
        C4 = input_batch[10:,10:,ll]
        C5 = input_batch[5:105,5:105,ll]

        C = [C1, C2, C3, C4, C5]

        for kk in range(5):
            #Rotate 4X more image (by 90 deg)
            for jj in range(4):
                C_R = np.rot90(C[kk], k=jj)
                input_batch_aug[:,:,count] = C_R
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1
                
                input_batch_aug[:,:,count] = np.swapaxes(C_R,0,1)
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1


    #PUT THE DATA AS A PYTORCH TENSOR:
    tensor_input_batch_aug = torch.Tensor(input_batch_aug)
    tensor_label_batch_aug = torch.Tensor(arr_label_batch_aug)
    
    return tensor_input_batch_aug, tensor_label_batch_aug


In [ ]:
#Use function defined above:
#rand_index = np.random.permutation(1403)
#rand_index = np.random.permutation(701)
rand_index = np.random.permutation(280)
#rand_index = np.random.permutation(140)
#shuffled numbers from 0 to 1402 (inclusive)
#goes up to 14,030 images, last 4 images load separatly or just ignore
#the random index goes up in steps of 10 in the function so randomizes the batching by selected groups of 10 images together

#Use this loop for training over entire dataset at each epochs
for ii in range(np.shape(rand_index)[0]):
  image_batch, label_batch = data_batch(datafile_index=50*rand_index[ii], num_images=50)
  #print(np.shape(image_batch), np.shape(label_batch))
  #print(ii)

#TAKES 40 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 10 IMAGES (400 when augmented)
#TAKES 21 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 20 IMAGES (800 when augmented)
#TAKES 12 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 50 IMAGES (2000 when augmented)
#TAKES 8 MIN TO LOAD ENTIRE DATASET FOR BATCH OF 100 IMAGES (4000 when augmented)

#USE 50 images for 12 MIN, good compromise for small enough batches and fast enoguh loading


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: ignored

In [ ]:
#Train and test set:
rand_index = np.random.permutation(280)

rand_train = rand_index[:200]
rand_test = rand_index[200:]

#Use this loop for training over entire dataset at each epochs
for ii in range(np.shape(rand_train)[0]):
  image_batch, label_batch = data_batch(datafile_index=50*rand_train[ii], num_images=50)

for ii in range(np.shape(rand_test)[0]):
  image_batch, label_batch = data_batch(datafile_index=50*rand_test[ii], num_images=50)


##Less Data Augmentation (only Cropping)

In [ ]:
#LESS DATA AUGMENTION: crop only = X5 augmentation:

#AUGMENT ONLY X5 (ONLY BY CROPPING)
def data_batch_aug5(datafile_index, num_images=10, data_file='/content/drive/MyDrive/data/data_g_band.txt'):
    '''
    Description:
        Access datafile.txt, each row is flattened 110x110 image + 1 label string (E, Sp, S0, Irr+Misc).
        Returns an augmented batch of num_images X 5.
        The labels are converted to 1D vectors (ex: Sp = [0,0,1,0])
        Need to give a datafile_index that tells which rows to pick.
    Inputs:
        datafile_index: index of row in datafile to load. loads rows datafile_index to datafile_index+num_images.
        num_images: number of different images to load per batch, total batch size 
        is 5 X num_images. (default: 10 (for 5X10 = 400 batch size like in paper)
        data_file: datafile full path, need to add shortcut to local Drive. (default: '/content/drive/MyDrive/data/data_g_band.txt')
    Outputs:
        tensor_input_batch_aug: dimensions: (100, 100, num_images X 5). 
        tensor_label_batch_aug: dimensions: (num_images X 5, 4)
    '''

    #data_file = 'data_g_band.txt'

    #Take batch of num_images rows from datafile:
    with open(data_file, 'r') as f:
        rows = f.readlines()[datafile_index:(datafile_index+num_images)]

    #for batch size of 400 (augmented), need 10 images
    data_batch = np.zeros((num_images,12101), dtype=np.dtype('U10'))
    count = 0
    for row in rows:
        data_batch[count,:] = row.split()
        count += 1

    #separate label and input:
    input_batch_flat = np.array(data_batch[:,:12100], dtype=int)
    label_batch = np.array(data_batch[:,-1])

    #convert input batch back to a 2D array:
    input_batch = np.empty((110,110,np.shape(input_batch_flat)[0]), dtype=int)
    for ii in range(np.shape(input_batch_flat)[0]):
        input_batch[:,:,ii] = np.reshape(input_batch_flat[ii,:], (110,110))


    #convert label batch into into 1D vector: 
    #E=0, S0=1, Sp=2, Irr+Misc=3
    #ex: label = [0,0,1,0] ==> Sp galagy
    arr_label_batch = np.empty((np.shape(label_batch)[0],4), dtype=int)
    arr_label_batch[:,0] = np.array([label_batch == 'E'], dtype=int)
    arr_label_batch[:,1] = np.array([label_batch == 'Sp'], dtype=int)
    arr_label_batch[:,2] = np.array([label_batch == 'S0'], dtype=int)
    arr_label_batch[:,3] = np.array([label_batch == 'Irr+Misc'], dtype=int)

    #test with image plotted
    #import matplotlib.pyplot as plt
    #plt.imshow(input_batch[:,:,0])
    #plt.show()

    #NOW AUGMENT THE BATCH (5X more):
    how_much_augment = 5
    input_batch_aug = np.empty((100,100,np.shape(input_batch)[2]*how_much_augment), dtype=int)
    arr_label_batch_aug = np.empty((np.shape(arr_label_batch)[0]*how_much_augment, 4), dtype=int)

    count = 0
    for ll in range(np.shape(input_batch)[2]):
        #Crop 5X more image (100X100 pixels)
        C1 = input_batch[:100,:100,ll]
        C2 = input_batch[10:,:100,ll]
        C3 = input_batch[:100,10:,ll]
        C4 = input_batch[10:,10:,ll]
        C5 = input_batch[5:105,5:105,ll]

        C = [C1, C2, C3, C4, C5]

        for kk in range(5):
            input_batch_aug[:,:,count] = C[kk]
            arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
            count += 1

    #PUT THE DATA AS A PYTORCH TENSOR:
    tensor_input_batch_aug = torch.Tensor(input_batch_aug)
    tensor_label_batch_aug = torch.Tensor(arr_label_batch_aug)
    
    return tensor_input_batch_aug, tensor_label_batch_aug


#Test above function:
rand_index = np.random.permutation(1403) #10 images
rand_train = rand_index[:200] #arbitrary values
rand_test = rand_index[200:]

#Use this loop for training over entire dataset at each epochs
for ii in range(np.shape(rand_train)[0]):
  image_batch, label_batch = data_batch_aug5(datafile_index=10*rand_train[ii], num_images=10)
  print(np.shape(image_batch))
  #print(np.shape(label_batch))
  #print(label_batch)
  #Check: 10 images X 5 augmentation = 100 x 100 x 50 tensor size
  #check: label size is 10 x 5 = 50 x 4 (4 labels)
  #check: label is 5 type in a row then another 5 in a row etc ...



torch.Size([100, 100, 50])
torch.Size([100, 100, 50])
torch.Size([100, 100, 50])


KeyboardInterrupt: ignored

##Data Augmentation for 3 Way Classification (not 4)

In [33]:
#Now with indexes of irregulars to skip, load 3 way classified data:

#LOAD THE DATA FROM TXT FILE INTO A BATCH:
def data_batch_3_way(datafile_index, num_images=10, data_file='/content/drive/MyDrive/data/data_g_band_3way.txt'):
    '''
    Description:
      3 way classifier data loader
    '''

    #Take batch of num_images rows from datafile:
    with open(data_file, 'r') as f:
        rows = f.readlines()[datafile_index:(datafile_index+num_images)]

    #for batch size of 400 (augmented), need 10 images
    data_batch = np.zeros((num_images,12101), dtype=np.dtype('U10'))
    count = 0
    for row in rows:
        data_batch[count,:] = row.split()
        count += 1

    #separate label and input:
    input_batch_flat = np.array(data_batch[:,:12100], dtype=int)
    label_batch = np.array(data_batch[:,-1])

    #convert input batch back to a 2D array:
    input_batch = np.empty((110,110,np.shape(input_batch_flat)[0]), dtype=int)
    for ii in range(np.shape(input_batch_flat)[0]):
        input_batch[:,:,ii] = np.reshape(input_batch_flat[ii,:], (110,110))


    #convert label batch into into 1D vector: 
    #E=0, S0=1, Sp=2, Irr+Misc=3
    #ex: label = [0,0,1,0] ==> Sp galagy
    arr_label_batch = np.empty((np.shape(label_batch)[0],3), dtype=int)
    arr_label_batch[:,0] = np.array([label_batch == 'E'], dtype=int)
    arr_label_batch[:,1] = np.array([label_batch == 'Sp'], dtype=int)
    arr_label_batch[:,2] = np.array([label_batch == 'S0'], dtype=int)
    
    #test with image plotted
    #import matplotlib.pyplot as plt
    #plt.imshow(input_batch[:,:,0])
    #plt.show()

    #NOW AUGMENT THE BATCH (40X more):
    input_batch_aug = np.empty((100,100,np.shape(input_batch)[2]*40), dtype=int)
    arr_label_batch_aug = np.empty((np.shape(arr_label_batch)[0]*40, 3), dtype=int)

    count = 0
    for ll in range(np.shape(input_batch)[2]):
        #Crop 5X more image (100X100 pixels)
        C1 = input_batch[:100,:100,ll]
        C2 = input_batch[10:,:100,ll]
        C3 = input_batch[:100,10:,ll]
        C4 = input_batch[10:,10:,ll]
        C5 = input_batch[5:105,5:105,ll]

        C = [C1, C2, C3, C4, C5]

        for kk in range(5):
            #Rotate 4X more image (by 90 deg)
            for jj in range(4):
                C_R = np.rot90(C[kk], k=jj)
                input_batch_aug[:,:,count] = C_R
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1
                
                input_batch_aug[:,:,count] = np.swapaxes(C_R,0,1)
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1


    #PUT THE DATA AS A PYTORCH TENSOR:
    tensor_input_batch_aug = torch.Tensor(input_batch_aug)
    tensor_label_batch_aug = torch.Tensor(arr_label_batch_aug)
    
    return tensor_input_batch_aug, tensor_label_batch_aug




#Train and test set:
rand_index = np.random.permutation(1360) #10 images #NOTE: 3 way ==> 1360, NOT 1403
rand_train = rand_index[:200]
rand_test = rand_index[200:]

#Use this loop for training over entire dataset at each epochs

for ii in range(np.shape(rand_train)[0]):
  image_batch, label_batch = data_batch_3_way(datafile_index=10*rand_train[ii], num_images=10)
  print(np.shape(image_batch), np.shape(label_batch))

for ii in range(np.shape(rand_test)[0]):
  image_batch, label_batch = data_batch(datafile_index=10*rand_test[ii], num_images=10)


torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100, 400]) torch.Size([400, 3])
torch.Size([100, 100

KeyboardInterrupt: ignored

In [ ]:
#Load All the data into into one large 'batch':

#LOAD THE DATA FROM TXT FILE INTO A BATCH:
def data_load_only_images(num_images=3, data_file='/content/drive/MyDrive/data/data_g_band.txt'):
    
    with open(data_file, 'r') as f:
        rows = f.readlines()[0:num_images]

    data_batch = np.zeros((num_images,12101), dtype=np.dtype('U10'))
    count = 0
    for row in rows:
        data_batch[count,:] = row.split()
        count += 1

    #separate label and input:
    input_batch_flat = np.array(data_batch[:,:12100], dtype=int)
    label_batch = np.array(data_batch[:,-1])

    #convert input batch back to a 2D array:
    input_batch = np.empty((110,110,np.shape(input_batch_flat)[0]), dtype=int)
    for ii in range(np.shape(input_batch_flat)[0]):
        input_batch[:,:,ii] = np.reshape(input_batch_flat[ii,:], (110,110))


    #convert label batch into into 1D vector: 
    #E=0, S0=1, Sp=2, Irr+Misc=3
    #ex: label = [0,0,1,0] ==> Sp galagy
    arr_label_batch = np.empty((np.shape(label_batch)[0],4), dtype=int)
    arr_label_batch[:,0] = np.array([label_batch == 'E'], dtype=int)
    arr_label_batch[:,1] = np.array([label_batch == 'Sp'], dtype=int)
    arr_label_batch[:,2] = np.array([label_batch == 'S0'], dtype=int)
    arr_label_batch[:,3] = np.array([label_batch == 'Irr+Misc'], dtype=int)

    #test with image plotted
    #import matplotlib.pyplot as plt
    #plt.imshow(input_batch[:,:,0])
    #plt.show()

    #NOW AUGMENT THE BATCH (40X more):
    input_batch_aug = np.empty((100,100,np.shape(input_batch)[2]*40), dtype=int)
    arr_label_batch_aug = np.empty((np.shape(arr_label_batch)[0]*40, 4), dtype=int)

    count = 0
    for ll in range(np.shape(input_batch)[2]):
        #Crop 5X more image (100X100 pixels)
        C1 = input_batch[:100,:100,ll]
        C2 = input_batch[10:,:100,ll]
        C3 = input_batch[:100,10:,ll]
        C4 = input_batch[10:,10:,ll]
        C5 = input_batch[5:105,5:105,ll]

        C = [C1, C2, C3, C4, C5]

        for kk in range(5):
            #Rotate 4X more image (by 90 deg)
            for jj in range(4):
                C_R = np.rot90(C[kk], k=jj)
                input_batch_aug[:,:,count] = C_R
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1
                
                input_batch_aug[:,:,count] = np.swapaxes(C_R,0,1)
                arr_label_batch_aug[count,:] = arr_label_batch[ll,:]
                count += 1


    #PUT THE DATA AS A PYTORCH TENSOR:
    tensor_input_batch_aug = torch.Tensor(input_batch_aug)
    tensor_label_batch_aug = torch.Tensor(arr_label_batch_aug)
    
    return tensor_input_batch_aug, tensor_label_batch_aug
